# **Project : A Case Study of InnovaTech Solutions**

**Business Overview:**

InnovaTech has expanded its presence in the digital retail world, especially on e-commerce giants like Amazon. This strategic move has not only widened its customer base but also resulted in a large influx of customer feedback, primarily in the form of online reviews. The company's products, notably its range of laptops, have become popular choices on these platforms, leading to an abundance of valuable but underutilized customer data.

**Current Challenge:**

InnovaTech currently analyzes customer reviews using basic sentiment analysis tools, which only provides a superficial understanding of customer opinions. In the competitive landscape of the laptop market, a more detailed and aspect-oriented analysis is crucial. Understanding specific customer sentiments on different aspects of laptops, such as user screen, technical specifications, etc, which is vital for targeted product improvements.

**Objective:**

The primary goal is to conduct a comprehensive aspect-based sentiment analysis of customer reviews for InnovaTech’s laptops, specifically focusing on three critical aspects: the laptop screen, keyboard, and mousepad. These components have been identified as crucial determinants of customer satisfaction and product usability. The project aims to provide nuanced insights into specific areas of customer satisfaction, dissatisfaction, and neutral feedback.The ultimate goal is to enhance overall product quality and customer experience, solidifying InnovaTech's position as a leader in the laptop market.

**Data Description:**

The dataset titled "laptop_reviews.csv" is structured to facilitate aspect-based sentiment analysis for laptop reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews
2. text: This column includes the actual text of the laptop reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the laptops.
3. aspects: Contains structured information about specific aspects mentioned in each review like 'RAM', 'screen', 'keyboard', 'mousepad', and others relevant to laptop features.
4. category: Provides an additional layer of classification (positive, negative and neutral) for the mentioned aspects.

**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  (2 Marks)

### Installation

In [ ]:
!pip install openai==1.2 tiktoken datasets session-info --quiet

### Imports

In [ ]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

from openai import AzureOpenAI
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np
import ast

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
session_info.show()

### Authentication

**(A) Writing/Creating the config.json file (2 Marks)**

In [ ]:
# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "<API_KEY>",            #Replace it with your credentials
    "AZURE_OPENAI_ENDPOINT": "<ENDPOINT>",      #Replace it with your credentials
    "AZURE_OPENAI_APIVERSION": "<API_VERSION>", #Replace it with your credentials
    "CHATGPT_MODEL": "<MODEL_NAME>"             #Replace it with your credentials
}

In [ ]:
# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

In [ ]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [ ]:
creds = json.loads(data)

In [ ]:
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [ ]:
chat_model_id = creds["CHATGPT_MODEL"]

### Utilities

In [ ]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

# Task: Aspect-Based Sentiment Analysis (ABSA)

**Step 2: Assemble Data (5 Marks)**

(A) Upload and Read csv File (1 Mark)

(B) Split the Dataset (2 Marks)

(C) Create a dictionary of aspects by creating two indexes - "examples_aspect_index" and "gold_examples_aspect_index" (1 Mark)

(D) Print the count of examples for each aspect in examples_aspect_index and gold_examples_aspect_index (1 mark)

**(A) Upload and read csv file (1 Mark)**

In [ ]:
laptop_reviews_df = "__________"
# Read CSV File Here

In [ ]:
# Check header of dataframe to examine data structure
laptop_reviews_df.head(5)

**(B) Split the Dataset (2 Marks)**

Now that the preprocessing is done, let us split the data into two segments (use split_ratio of 0.2) - one segment (80%) that gives us a pool to draw few-shot examples from and another segment (20%) that gives us a pool of gold examples.

In [ ]:
laptop_reviews_examples_df, laptop_reviews_gold_examples_df = train_test_split(
    "__________", #<- the full dataset
    "__________", #<- 20% random sample selected for gold examples
    random_state=42 #<- ensures that the splits are the same for every session
)

**(C) Create a dictionary of aspects by creating two indexes - "examples_aspect_index" and "gold_examples_aspect_index"** (1 Mark)

In [ ]:
examples_aspect_index = {
    '________': [],
    '________': [],
    '________': []
}

gold_examples_aspect_index = {
    '________': [],
    '________': [],
    '________': []
}

**(D) Print the count of examples for each aspect in examples_aspect_index and gold_examples_aspect_index (1 mark)**

In [ ]:
for id, category in zip(laptop_reviews_examples_df.id, laptop_reviews_examples_df.category):
    for key in examples_aspect_index.keys():
        if key in category:
            examples_aspect_index[key].append(id)

In [ ]:
# for examples_aspect_index
for key in __________:                            
    print(f"Number of examples for aspect {key}: {len(__________[key])}")

In [ ]:
for id, category in zip(laptop_reviews_gold_examples_df.id, laptop_reviews_gold_examples_df.category):
    for key in gold_examples_aspect_index.keys():
        if key in category:
            gold_examples_aspect_index[key].append(id)

In [ ]:
# for gold_examples_aspect_index
for key in __________:                            
    print(f"Number of examples for aspect {key}: {len(__________[key])}")

In [ ]:
columns_to_select = ['id', 'text', 'category']

In [ ]:
gold_examples = json.loads((
        laptop_reviews_gold_examples_df.loc[:, columns_to_select]
                                           .sample(15, random_state=42) #<- ensures that gold examples are the same for every session
                                           .to_json(orient='records')
))

In [ ]:
gold_examples[0]

**Step 3: Derive Prompt (12 Marks)**

(A) Write Zero Shot System Message (3 Marks)

(B) Create Zero Shot Prompt (2 Marks)

(C) Write Few Shot System Message (3 Marks)

(D) Create Examples For Few shot prompte (2 Marks)

(E) Create Few Shot Prompt (2 Marks)

In [ ]:
user_message_template = """```{laptop_review}```"""

**(A) Write Zero Shot System Message (3 Marks)**

In [ ]:
zero_shot_system_message = """__________"""
# Write zero shot system message here

**(B) Create Zero Shot Prompt (2 Marks)**

In [ ]:
zero_shot_prompt = "__________"
# Create zero shot prompt to be input ready for completion function

In [ ]:
num_tokens_from_messages(zero_shot_prompt)

**(C) Write Few Shot System Message (3 Marks)**

In [ ]:
few_shot_system_message = """__________"""

In [ ]:
def create_examples(dataset, n=4):

    """
    Return a JSON list with n random examples of each aspect in the
    input dataset.
    First create a dictionary with the aspects as keys and the ids of the
    reviews that contain this aspect as values.
    Then take a random sample of ids from each of these lists.

    Args:
        dataset (DataFrame): DataFrame with nested ABSA annotations
        n (int): Number of random examples selected for each aspect

    Output:
        examples (JSON): JSON list of examples
    """

    columns_to_select = ['id', 'text', 'category']
    example_ids = []

    aspect_index = {
        'screen': [], 'keyboard': [], 'mousepad': []
    }

    for id, category in zip(dataset.id, dataset.category):
        for key in aspect_index.keys():
            if key in category:
                aspect_index[key].append(id)

    for key in aspect_index:
        example_ids.extend(np.random.choice(aspect_index[key], n).tolist())

    examples = dataset.loc[dataset.id.isin(example_ids), columns_to_select]

    return examples.to_json(orient='records')

**(D) Create Examples For Few shot prompte (2 Marks)**

In [ ]:
examples = "__________"
# Create Examples

In [ ]:
json.loads(examples)

With the examples in place, we can now assemble a few-shot prompt. Since we will be using the few-shot prompt several times during evaluation, let us write a function to create a few-shot prompt (the logic of this function is depicted below).

In [ ]:
def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): Instructions for the model to execute ABSA
        examples (JSON): JSON list of examples representative of each aspect
        user_message_template (str): string with a placeholder for laptop reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': system_message}]

    for example in json.loads(examples):
        example_input = example['text']
        example_absa = example['category']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content': user_message_template.format(
                    laptop_review=example_input
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant', 'content': f"{example_absa}"}
        )

    return few_shot_prompt

**(E) Create Few Shot Prompt (2 Marks)**

In [ ]:
few_shot_prompt = "__________"
# Create Few shot prompt

In [ ]:
few_shot_prompt

In [ ]:
num_tokens_from_messages(few_shot_prompt)

**Step 4: Evaluate prompts (8 Marks)**

(A) Evaluate Zero Shot Prompt (2 Marks)

(B) Evaluate Few Shot Prompt (2 marks)

(C) Calculate Mean and Standard Deviation for Few Shot Prompt (4 Marks)

Now we have two sets of prompts that we need to evaluate using gold labels. Since the few-shot prompt depends on the sample of examples that was drawn to make up the prompt, we expect some variability in evaluation. Hence, we evaluate each prompt multiple times to get a sense of the average and the variation around the average.

To reiterate, a choice on the prompt should account for variability due to the choice of the random sample. To aid repeated evaluation, we assemble an evaluation function .

In [ ]:
def custom_parse(string):
    result = {}
    # Remove the outermost curly braces and split by ', "'
    parts = string.strip('{}').split(', "')

    for part in parts:
        # Split each part by the first occurrence of '": '
        split_index = part.find('": ')
        if split_index == -1:
            continue  # Skip if the format is not as expected

        key = part[:split_index].strip('"')
        value = part[split_index+3:].strip()

        # Handle the array and extract the first element
        if value.startswith('array(['):
            value = value[7:]  # Remove 'array(['
            value = value.split('], dtype=object')[0]  # Get the first element
            value = value.strip('["]')  # Remove quotes and brackets

        result[key] = value

    return result

def compute_accuracy(gold_examples, model_predictions, ground_truths):
    correct_predictions = 0
    total_predictions = len(gold_examples)

    for pred, truth in zip(model_predictions, ground_truths):
        pred_dict = custom_parse(pred)
        truth_dict = custom_parse(truth)

        if pred_dict == truth_dict:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

def evaluate_prompt(prompt, gold_examples, user_message_template):
    model_predictions, ground_truths = [], []

    for example in gold_examples:
        user_input = [{
            'role': 'user',
            'content': user_message_template.format(laptop_review=example['text'])
        }]

        try:
            response = client.chat.completions.create(
                model=chat_model_id,
                messages=prompt + user_input,
                temperature=0,
                #max_tokens=2
            )
            
            prediction = response.choices[0].message.content

            #prediction = response['choices'][0]['message']['content']
            # Convert to string representation of dictionary
            prediction_dict_str = str(prediction).replace("'", "\"")
            print(f"Model Prediction (Before Parsing): {prediction_dict_str}")

            model_predictions.append(prediction_dict_str.strip().lower())
            ground_truth_str = str(example['category']).replace("'", "\"")
            print(f"Ground Truth (Before Parsing): {ground_truth_str}")

            ground_truths.append(ground_truth_str.strip().lower())

        except Exception as e:
            print(f"Error during model prediction: {e}")

    accuracy = compute_accuracy(gold_examples, model_predictions, ground_truths)
    return accuracy

Let us now use this function to do one evaluation of all the two prompts assembled so far, each time computing the accuracy score.

**(A) Evaluate zero shot prompt (2 Marks)**

In [ ]:
"_________"

**(B) Evaluate few shot prompt (2 Marks)**

In [ ]:
"_________"

However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 10 times.

In [ ]:
num_eval_runs = 10

In [ ]:
few_shot_performance = []

In [ ]:
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(laptop_reviews_df)

    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

    # Evaluate prompt accuracy on gold examples
    few_shot_accuracy = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    few_shot_performance.append(few_shot_accuracy)

**(C) Calculate Mean and Standard Deviation for Few Shot Prompt (4 Marks)**

Compute the average (mean) and measure the variability (standard deviation) of the evaluation score for few shot prompt.

In [ ]:
"__________"
# Calculate for Few Shot Prompt

**Step 5: Observation and Insights and Business perspective (3 Marks)**

( Based on the projects, learner needs to share observations, learnings, insights and the business use case where these learnings can be beneficial.
Provide a breakdown of the percentage of positive and negative reviews. Additionally, explain how this classification can assist InnovaTech Solutions in addressing the issues identified. )


**-------------------------------------------------------------------------------------------------------End-------------------------------------------------------------------------------------------------------**